# Monorail.json生成ファイル
GoogleDrive上の共有フォルダに配置されたモノレールのデータをもとに、monorail.jsonを作成する

In [1]:
!python --version

Python 3.6.9


## Google Drive APIに接続

In [2]:
import os
import json

pk=os.environ['SERVICE_ACCOUNT_JSON']

with open('service_account.json','w') as f:
    json_loads = json.loads(pk,strict=False)
    json.dump(json_loads, fp=f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))

In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
credentials = service_account.Credentials.from_service_account_file('service_account.json')
scoped_credentials = credentials.with_scopes(
  [
    'https://www.googleapis.com/auth/drive'
  ])

service = build('drive', 'v3', credentials=credentials)

## 共有フォルダ「コロナ対策サイト」の「10_千葉市受領データ」配下のフォルダを検索

In [4]:
import io
import pandas as pd

results = service.files().list(
    corpora="drive",
    driveId='0AD0xvJ3ujNNmUk9PVA',
    includeItemsFromAllDrives=True,
    supportsAllDrives=True,
    orderBy='createdTime desc',
    q=f"'1mK3sX_7Kx3T4FhFG_NbBnIQLhtdHG16K' in parents and mimeType = 'application/vnd.google-apps.folder'",
    pageSize=100, fields="nextPageToken, files(id, name, mimeType, createdTime)").execute()
items = results.get('files', [])
df=pd.DataFrame(items)
df.columns = ['id', 'name', 'mimeType' ,'createdTime']
df.head()

,id,name,mimeType,createdTime
0,12sOJRki837z57RguTELlGOfhljkrSlXI,20210706,application/vnd.google-apps.folder,2021-07-06T04:35:59.556Z
1,1Xj52-ecpITOo-FHz7uaGDN9xKFj9Leyw,20210628,application/vnd.google-apps.folder,2021-06-28T07:39:41.247Z
2,1hUg25adPpuigDscHLoyc3orIsdqjG7Bu,20210623,application/vnd.google-apps.folder,2021-06-23T00:38:59.982Z
3,1w2zwDRpf3tf4e0vrPMNlrS7fsbXKugq-,20210615,application/vnd.google-apps.folder,2021-06-15T04:30:23.180Z
4,1wMTq6-Y5m9FB5KCqjTgtZDmM6p5Wyl4l,20210609,application/vnd.google-apps.folder,2021-06-09T06:19:15.363Z


## 更新日付の最も新しいフォルダ内にxlsx形式のファイルがあるか検索

In [5]:
item = next(iter(items))

results = service.files().list(
    corpora="drive",
    driveId='0AD0xvJ3ujNNmUk9PVA',
    includeItemsFromAllDrives=True,
    supportsAllDrives=True,
    q=f"'{item['id']}' in parents and mimeType ='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'",
    pageSize=100, fields="nextPageToken, files(id, name, mimeType, createdTime)").execute()
items = results.get('files', [])

#一番先頭のフォルダの内容を表示
item = next(iter(items))
df=pd.DataFrame(items)
df.columns = ['id', 'name', 'mimeType' ,'createdTime']
df.head()

,id,name,mimeType,createdTime
0,1xkFUyjVnQzVVjOZIB9_bN52jmqO_SRb-,モノレール利用者数データ（千葉駅）.xlsx,application/vnd.openxmlformats-officedocument....,2021-07-06T04:36:41.031Z


## モノレール利用者数データを取得し、カレントフォルダに出力

In [6]:
from googleapiclient.http import MediaIoBaseDownload

request = service.files().get_media(fileId=item['id'])
fh = io.FileIO('モノレール利用者数データ（千葉駅）.xlsx', "wb")
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()

In [7]:
ls

README.md      dummy.json
composer.json  make_monorail_json.ipynb
composer.lock  requirements.txt
convert.php    service_account.json
downloads/     モノレール利用者数データ（千葉駅）.xlsx


In [8]:
import datetime as dt

CREATED_DATE=dt.datetime.today().strftime('%Y/%m/%d %H:%M')
CREATED_DATE

'2021/07/07 20:33'

In [9]:
import pandas as pd

# データ読み込み
df = pd.read_excel('./モノレール利用者数データ（千葉駅）.xlsx',sheet_name='Sheet2',header=1,usecols=list(range(0,11)),engine='openpyxl')
df.columns = ['開始日','終了日', '日数' ,'06:30:00' ,'07:00:00' ,'07:30:00' ,'08:00:00' ,'08:30:00' ,'09:00:00' ,'09:30:00' ,'10:00:00']

# 2.「01_monorail_jsonを生成.ipynb」に対する修正内容
# 開始日、終了日を次のフォーマットに変換(MM月DD日)
# 期間を次の内容で作成。「開始日」+「～」＋「終了日」
df['開始日'] = pd.to_datetime(df['開始日']).dt.strftime('%-m月%-d日')
df['終了日'] = pd.to_datetime(df['終了日']).dt.strftime('%-m月%-d日')
df['期間'] = df['開始日'] + '～' + df['終了日']

df = df.dropna()
# df = df.set_index('期間')

# 各時間帯に整理
df['6:30～7:30'] = (df['06:30:00']+ df['07:00:00'])/df['日数']
df['7:30～9:30'] = (df['07:30:00']+ df['08:00:00'] + df['08:30:00']+ df['09:00:00'])/df['日数']
df['9:30～10:30'] = (df['09:30:00']+ df['10:00:00'])/df['日数']

# パーセントにする
base_date_index = 0
df['p_6:30～7:30'] = df['6:30～7:30']/df.at[base_date_index, '6:30～7:30']
df['p_7:30～9:30'] = df['7:30～9:30']/df.at[base_date_index, '7:30～9:30']
df['p_9:30～10:30'] = df['9:30～10:30']/df.at[base_date_index, '9:30～10:30']

# 減少率にする
df['p_6:30～7:30'] = df['p_6:30～7:30'] - 1.0
df['p_7:30～9:30'] = df['p_7:30～9:30'] - 1.0
df['p_9:30～10:30'] = df['p_9:30～10:30'] - 1.0

# パーセントの値にする
df['p_6:30～7:30'] = (df['p_6:30～7:30']*100).round(2)
df['p_7:30～9:30'] = (df['p_7:30～9:30']*100).round(2)
df['p_9:30～10:30'] = (df['p_9:30～10:30']*100).round(2)

In [10]:
df

,開始日,終了日,日数,06:30:00,07:00:00,07:30:00,08:00:00,08:30:00,09:00:00,09:30:00,10:00:00,期間,6:30～7:30,7:30～9:30,9:30～10:30,p_6:30～7:30,p_7:30～9:30,p_9:30～10:30
0,1月20日,1月24日,5.0,2258.0,2736.0,3288.0,2719.0,2637.0,1519.0,1496.0,1523.0,1月20日～1月24日,998.8,2032.60,603.8,0.00,0.00,0.00
1,1月27日,1月31日,5.0,2316.0,2821.0,3264.0,2698.0,2468.0,1526.0,1408.0,1411.0,1月27日～1月31日,1027.4,1991.20,563.8,2.86,-2.04,-6.62
2,2月3日,2月7日,5.0,2271.0,2800.0,3198.0,2512.0,2483.0,1574.0,1404.0,1422.0,2月3日～2月7日,1014.2,1953.40,565.2,1.54,-3.90,-6.39
3,2月10日,2月14日,4.0,1763.0,2225.0,2554.0,2047.0,1954.0,1208.0,1200.0,1208.0,2月10日～2月14日,997.0,1940.75,602.0,-0.18,-4.52,-0.30
4,2月17日,2月21日,5.0,2234.0,2763.0,3196.0,2459.0,2408.0,1546.0,1513.0,1355.0,2月17日～2月21日,999.4,1921.80,573.6,0.06,-5.45,-5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,5月31日,6月4日,5.0,1733.0,2337.0,2969.0,1986.0,2147.0,1308.0,1197.0,1043.0,5月31日～6月4日,814.0,1682.00,448.0,-18.50,-17.25,-25.80
72,6月7日,6月11日,5.0,1683.0,2347.0,2867.0,2037.0,2102.0,1266.0,1217.0,1089.0,6月7日～6月11日,806.0,1654.40,461.2,-19.30,-18.61,-23.62
73,6月14日,6月18日,5.0,1702.0,2339.0,2856.0,2086.0,2122.0,1217.0,1328.0,1161.0,6月14日～6月18日,808.2,1656.20,497.8,-19.08,-18.52,-17.56
74,6月21日,6月25日,5.0,1721.0,2367.0,2860.0,2072.0,2092.0,1186.0,1367.0,1255.0,6月21日～6月25日,817.6,1642.00,524.4,-18.14,-19.22,-13.15


In [11]:
# データをjsonの形式加工する
df3=df[['期間','p_6:30～7:30','p_7:30～9:30','p_9:30～10:30']].rename(columns={
'p_6:30～7:30':'6:30～7:30',
'p_7:30～9:30':'7:30～9:30',
'p_9:30～10:30':'9:30～10:30',
})

dataset_list = []
for row in df3.drop(index=base_date_index).to_dict(orient='records'):
  #  {'6:30～7:30': -29.52,
  #   '7:30～9:30': -28.36,
  #   '9:30～10:30': -25.22,
  #   '期間': '01月11日~01月15日'},
  #   ↓
  #   {'data': ['02月03日~02月07日', 1.54, -3.9, -6.39], 'label': 2},
  _label = row['期間']
  _data = [row['6:30～7:30'],row['7:30～9:30'],row['9:30～10:30']]
  dataset_list.append(dict(data=_data,label=_label))

json_data = {
  "date": CREATED_DATE,
  "datasets": dataset_list,
  "labels": ["6:30~7:30", "7:30~9:30", "9:30~10:30"],
  "base_period": "1月20日~1月24日"
}

In [12]:
import json
with open(f'monorail.json','w') as f:
  json.dump(json_data, fp=f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))